# Data Gathering and Data Cleaning
#### By: _Zhan Yu_

## Table of Contents
- [Loading Libraries & Data](#Loading-Libraries-&-Data)

## Loading Libraries & Data

In this project, the two datasets [Rodent Inspection in NYC](https://data.cityofnewyork.us/Health/Rodent-Inspection/p937-wjvj) and [Restaurant-Inspection](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j ) are public datasets from data.cityofnewyork.us

In [1]:
# Libraries: 
import pandas as pd
import numpy as np
import os

from sodapy import Socrata

import warnings
warnings.simplefilter(action="ignore")

### SoQL(Socrata Query Language)  

The Socrata APIs provide rich query functionality through a query language we call the “Socrata Query Language” or “SoQL”.   
Install packages at the current environment (for example, mine is (dsi)) before running:
``` Terminal
pip install sodapy
```

In [2]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# In this case they are public datasets.

# First 1,000,000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("p937-wjvj",                                # Rodent Inspection dataset
                     limit=1_000_000, where="boro_code = 1")

# Convert to pandas DataFrame
rats = pd.DataFrame.from_records(results)

In [3]:
# First 500,000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("43nn-pn8j",                                # NYC Restaurant Inspection dataset
                     limit=500_000)

# Convert to pandas DataFrame
restaurants = pd.DataFrame.from_records(results)

Because the two datasets are too big (both about 200 MB), we are going to do some data cleaning and trimming before we export them.

## Data Cleaning

### rats.csv  

In [4]:
# Setting the index to 'job_id':
rats.set_index('job_id',inplace = True)
rats.head()

,inspection_type,job_ticket_or_work_order_id,job_progress,bbl,boro_code,block,lot,house_number,street_name,zip_code,x_coord,y_coord,latitude,longitude,borough,inspection_date,result,approved_date,location
job_id,,,,,,,,,,,,,,,,,,,
PO2075886,INITIAL,2075886,1,1022430001,1,02243,0001,4977,BROADWAY,10034,1006543,255608,40.868203550611,-73.919469362846,Manhattan,2019-08-02T11:11:55.000,Active Rat Signs,2019-08-07T11:34:09.000,"{'latitude': '40.868203550611', 'longitude': '..."
PO333717,BAIT,70581,5,1009620100,1,00962,0100,462,1 AVENUE,10016,991147,208600,40.739489790595,-73.976623458224,Manhattan,2011-10-26T09:45:58.000,Bait applied,2011-10-28T07:14:07.000,"{'latitude': '40.739489790595', 'longitude': '..."
PO431003,BAIT,92841,1,1022290001,1,02229,0001,4986,BROADWAY,10034,1006611,255630,40.868266491555,-73.919201730342,Manhattan,2012-03-19T14:00:03.000,Bait applied,2012-03-20T07:20:09.000,"{'latitude': '40.868266491555', 'longitude': '..."
PO333717,BAIT,69386,4,1009620100,1,00962,0100,462,1 AVENUE,10016,991147,208600,40.739489790595,-73.976623458224,Manhattan,2011-10-17T13:50:37.000,Bait applied,2011-10-18T07:09:09.000,"{'latitude': '40.739489790595', 'longitude': '..."
PO422674,BAIT,91531,1,1022290001,1,02229,0001,4986,BROADWAY,10034,1006611,255630,40.868266491555,-73.919201730342,Manhattan,2012-03-08T13:51:50.000,Bait applied,2012-03-09T12:23:58.000,"{'latitude': '40.868266491555', 'longitude': '..."


In [5]:
# Check the shape of data frame:
rats.shape

(594536, 19)

After taking a first look at our dataset, we have a general idea of features of dataset. We are going to only keep the features we need.

In [6]:
rats_df = rats[['inspection_type',
                'job_progress', 'bbl','latitude',
                'longitude','inspection_date', 'result', 'approved_date']]
rats_df.head()

,inspection_type,job_progress,bbl,latitude,longitude,inspection_date,result,approved_date
job_id,,,,,,,,
PO2075886,INITIAL,1,1022430001,40.868203550611,-73.919469362846,2019-08-02T11:11:55.000,Active Rat Signs,2019-08-07T11:34:09.000
PO333717,BAIT,5,1009620100,40.739489790595,-73.976623458224,2011-10-26T09:45:58.000,Bait applied,2011-10-28T07:14:07.000
PO431003,BAIT,1,1022290001,40.868266491555,-73.919201730342,2012-03-19T14:00:03.000,Bait applied,2012-03-20T07:20:09.000
PO333717,BAIT,4,1009620100,40.739489790595,-73.976623458224,2011-10-17T13:50:37.000,Bait applied,2011-10-18T07:09:09.000
PO422674,BAIT,1,1022290001,40.868266491555,-73.919201730342,2012-03-08T13:51:50.000,Bait applied,2012-03-09T12:23:58.000


In [7]:
rats_df.isnull().mean()

inspection_type    0.000000
job_progress       0.000000
bbl                0.000000
latitude           0.000891
longitude          0.000891
inspection_date    0.000000
result             0.000002
approved_date      0.000000
dtype: float64

Since the missing values have very small percentage of total dataset, we can drop all the rows with missing values and still have a relatively large dataset.

In [8]:
rats_df.dropna(inplace = True)
rats_df.shape

(594005, 8)

In [9]:
rats_df.isnull().sum().sum()

0

We are going to check and change the original data types:

In [10]:
# Checking the original data types:
rats_df.dtypes

inspection_type    object
job_progress       object
bbl                object
latitude           object
longitude          object
inspection_date    object
result             object
approved_date      object
dtype: object

In [11]:
# Changing date columns into datetime form: 
rats_df['inspection_date'] = pd.to_datetime(rats_df['inspection_date'])
rats_df['approved_date'] = pd.to_datetime(rats_df['approved_date'])

In [12]:
# Changing those columns which seem like numerical but actually strings:
rats_df['job_progress']=rats_df['job_progress'].astype(int)
rats_df['bbl']=rats_df['bbl'].astype(int)
rats_df['latitude']=rats_df['latitude'].astype(float)
rats_df['longitude']=rats_df['longitude'].astype(float)

In [13]:
# Checking data types again:
rats_df.dtypes

inspection_type            object
job_progress                int64
bbl                         int64
latitude                  float64
longitude                 float64
inspection_date    datetime64[ns]
result                     object
approved_date      datetime64[ns]
dtype: object

We are setting out dataset in chronological order because the time range of our data set is than 100 years.

In [14]:
set(rats_df['inspection_type'])

{'BAIT', 'CLEAN_UPS', 'COMPLIANCE', 'INITIAL'}

In [15]:
set(rats_df['result'])

{'Active Rat Signs',
 'Bait applied',
 'Cleanup done',
 'Monitoring visit',
 'Passed Inspection',
 'Problem Conditions'}

In [16]:
rats_df = rats_df.sort_values(by = 'inspection_date', ascending = False)
rats_df.head()

,inspection_type,job_progress,bbl,latitude,longitude,inspection_date,result,approved_date
job_id,,,,,,,,
PO2167036,COMPLIANCE,2,1016390133,40.795054,-73.943041,2020-01-08 13:20:54,Active Rat Signs,2020-01-09 08:16:29
PO2167008,COMPLIANCE,2,1016390021,40.795249,-73.943500,2020-01-08 13:10:44,Active Rat Signs,2020-01-09 08:16:29
PO2189724,INITIAL,1,1004160001,40.720992,-73.990498,2020-01-08 13:10:40,Passed Inspection,2020-01-09 09:39:57
PO2125410,COMPLIANCE,2,1010100046,40.765455,-73.977827,2020-01-08 13:05:13,Passed Inspection,2020-01-09 08:54:36
PO2167016,COMPLIANCE,2,1016390035,40.794941,-73.942221,2020-01-08 13:00:09,Passed Inspection,2020-01-09 08:16:29


Let's only see the last 10 years data which are still the majority of data:

In [17]:
rats_df = rats_df.loc[rats_df['inspection_date'] > '2010-01-01']
rats_df.shape

(591855, 8)

In [27]:
rats_df = rats_df.drop(rats_df.loc[rats_df['latitude'] == 0.0].index)
rats_df = rats_df.drop(rats_df.loc[rats_df['longitude'] == 0.0].index)

In [29]:
rats_df['longitude'].describe()

count    589467.000000
mean        -73.967300
std           0.023675
min         -74.018276
25%         -73.987864
50%         -73.966999
75%         -73.946232
max         -73.906864
Name: longitude, dtype: float64

In [30]:
rats_df['latitude'].describe()

count    589467.000000
mean         40.776468
std           0.042192
min          40.700774
25%          40.734885
50%          40.778963
75%          40.806505
max          40.878086
Name: latitude, dtype: float64

#### Export the cleaned data:

In [31]:
rats_df.to_csv('../datasets/rats.csv', index=False)

In [32]:
rats_df['bbl'].value_counts().head(10)

1003020001    428
1001650001    408
1004200001    348
1001220001    327
1003890036    313
1015240046    223
1020820047    216
1003760007    214
1003740041    210
1001690001    197
Name: bbl, dtype: int64

In [33]:
top10 = pd.DataFrame(index = rats_df['bbl'].value_counts().head(10).index, columns = ['latitude', 'longitude'])
for i in top10.index:
    top10.loc[i]['latitude'] = np.unique(rats_df.loc[rats_df['bbl'] == i]['latitude'])
    top10.loc[i]['longitude'] = np.unique(rats_df.loc[rats_df['bbl'] == i]['longitude'])
top10

,latitude,longitude
1003020001,[40.718008702954],[-73.99342724941]
1001650001,[40.715450766367],[-73.999538265827]
1004200001,"[40.718274963396, 40.719784495498, 40.72163087...","[-73.993748286251, -73.992269055289, -73.99158..."
1001220001,[40.713806515614],[-74.005569530537]
1003890036,"[40.723545956581, 40.723680527621, 40.72376550...","[-73.979692000943, -73.979259113526, -73.97910..."
1015240046,[40.785967020536],[-73.951206362211]
1020820047,[40.829347984541],[-73.945645775795]
1003760007,"[40.7239438899, 40.72402347752]","[-73.978941506339, -73.978883756867]"
1003740041,"[40.721292227281, 40.72146797746]","[-73.978231589502, -73.977787910393]"
1001690001,[40.715917367248],[-74.001565578432]


In [34]:
grand_st = rats_df.loc[rats_df['bbl']== 1003020001].drop(columns = ['bbl'])
grand_st.shape

(428, 7)

In [40]:
grand_st.set_index('inspection_date',inplace = True)

,inspection_type,job_progress,latitude,longitude,result,approved_date
inspection_date,,,,,,
2019-11-18 14:40:34,INITIAL,1,40.718009,-73.993427,Passed Inspection,2019-11-19 13:38:14
2019-10-23 16:41:06,INITIAL,1,40.718009,-73.993427,Active Rat Signs,2019-10-24 12:12:51
2019-09-18 15:15:50,INITIAL,1,40.718009,-73.993427,Active Rat Signs,2019-09-19 11:03:46
2019-08-26 15:16:25,INITIAL,1,40.718009,-73.993427,Active Rat Signs,2019-08-27 13:01:51
2019-07-25 15:16:16,INITIAL,1,40.718009,-73.993427,Active Rat Signs,2019-07-29 08:39:23
...,...,...,...,...,...,...
2012-05-10 14:34:56,INITIAL,1,40.718009,-73.993427,Active Rat Signs,2012-05-21 15:38:26
2011-04-29 11:04:50,INITIAL,1,40.718009,-73.993427,Active Rat Signs,2011-04-29 14:50:37
2011-04-15 15:31:16,INITIAL,1,40.718009,-73.993427,Active Rat Signs,2011-04-19 12:39:18


In [35]:
# Export data: 
grand_st.to_csv('../datasets/grand_st.csv', index=False)

In [36]:
columbus_pk = rats_df.loc[rats_df['bbl']== 1001650001].drop(columns = ['bbl'])
columbus_pk.shape

(408, 7)

In [37]:
# Export data: 
columbus_pk.to_csv('../datasets/columbus_pk.csv', index=False)

**restaurants.csv**

In [41]:
restaurants.shape

(401346, 26)

In [42]:
restaurants = restaurants.loc[restaurants['boro'] == 'Manhattan']

In [43]:
restaurants.head()

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,...,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
1,40868400,ARIANA KEBAB HOUSE,Manhattan,787,9 AVENUE,10019,2122622323,Afghan,2019-08-20T00:00:00.000,Violations were cited in the following area(s).,...,2020-03-02T06:01:15.000,Cycle Inspection / Initial Inspection,40.764993098016,-73.987855987299,104,03,013300,1082418,1010620032,MN15
2,50064218,TAP NYC,Manhattan,267,COLUMBUS AVE,10023,9175957012,Café/Coffee/Tea,2018-10-10T00:00:00.000,Violations were cited in the following area(s).,...,2020-03-02T06:01:15.000,Cycle Inspection / Initial Inspection,40.77771642487,-73.978556145725,107,06,015700,1028631,1011250001,MN14
8,41397650,"DUNKIN', BASKIN ROBBINS",Manhattan,360,WEST 31 STREET,10001,2129041160,Donuts,2019-04-26T00:00:00.000,Violations were cited in the following area(s).,...,2020-03-02T06:01:15.000,Cycle Inspection / Initial Inspection,40.750970713268,-73.996491829355,104,03,010300,1013540,1007540078,MN13
12,41271483,KATSUHAMA RESTAURANT,Manhattan,4345,WEST 55 STREET,NaN,2125417145,Japanese,2018-04-19T00:00:00.000,Violations were cited in the following area(s).,...,2020-03-02T06:01:15.000,Cycle Inspection / Re-inspection,0,0,NaN,NaN,NaN,NaN,1,NaN
14,50051737,SPRING PLACE,Manhattan,6,SAINT JOHNS LN,10013,2122575620,Mediterranean,2016-08-30T00:00:00.000,Violations were cited in the following area(s).,...,2020-03-02T06:01:15.000,Pre-permit (Operational) / Initial Inspection,40.720720569178,-74.005916438676,101,01,003300,1002734,1002127504,MN24


In [7]:
restaurants.isnull().mean()

camis                    0.000000
dba                      0.000811
boro                     0.000000
building                 0.000665
street                   0.000000
zipcode                  0.016704
phone                    0.000000
cuisine_description      0.000000
inspection_date          0.000000
action                   0.003598
violation_code           0.013454
violation_description    0.020105
critical_flag            0.020105
score                    0.042313
grade                    0.497169
grade_date               0.504136
record_date              0.000000
inspection_type          0.003598
latitude                 0.001051
longitude                0.001051
community_board          0.017755
council_district         0.017679
census_tract             0.017679
bin                      0.020764
bbl                      0.001051
nta                      0.017755
dtype: float64

In [8]:
restaurants.head(2)

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,...,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
1,40400049,PANYA/AUTRE KYOYA,Manhattan,810,STUYVESANT STREET,NaN,2125980454,Japanese,2017-01-03T00:00:00.000,Violations were cited in the following area(s).,...,2020-02-25T06:00:52.000,Cycle Inspection / Re-inspection,0,0,NaN,NaN,NaN,NaN,1,NaN
3,41279957,LELABAR,Manhattan,422,HUDSON STREET,10014,2122060594,American,2017-05-05T00:00:00.000,Violations were cited in the following area(s).,...,2020-02-25T06:00:52.000,Cycle Inspection / Initial Inspection,40.730478164589,-74.006801296496,102,03,006700,1009774,1005830002,MN23


In [22]:
restaurants['bbl'].value_counts()

1             3112
1012800001     520
1007130001     432
1007810002     391
1000160125     255
              ... 
1003900036       1
1001750029       1
1004680039       1
1006620003       1
1004360042       1
Name: bbl, Length: 7136, dtype: int64

In [45]:
set(restaurants['grade'])

{'A', 'B', 'C', 'G', 'N', 'P', 'Z', nan}

In [16]:
grad_a = restaurants.loc[restaurants['grade']=='A'].loc[restaurants['action']=='No violations were recorded at the time of this inspection.']

In [17]:
grad_a.shape

(283, 26)

In [20]:
grad_a.isnull().sum()

camis                      0
dba                        0
boro                       0
building                   1
street                     0
zipcode                   15
phone                      0
cuisine_description        0
inspection_date            0
action                     0
violation_code           283
violation_description    283
critical_flag            283
score                      0
grade                      0
grade_date                 0
record_date                0
inspection_type            0
latitude                   1
longitude                  1
community_board           16
council_district          16
census_tract              16
bin                       16
bbl                        1
nta                       16
dtype: int64

In [46]:
set(restaurants['action'])

{'Establishment Closed by DOHMH.  Violations were cited in the following area(s) and those requiring immediate action were addressed.',
 'Establishment re-closed by DOHMH',
 'Establishment re-opened by DOHMH',
 'No violations were recorded at the time of this inspection.',
 'Violations were cited in the following area(s).',
 nan}

In [47]:
set(restaurants['violation_description'])

{'""No Smoking” and/or \'Smoking Permitted” sign not conspicuously posted. Health warning not present on \'Smoking Permitted”',
 '""Wash hands” sign not posted at hand wash facility.',
 'A food containing artificial trans fat, with 0.5 grams or more of trans fat per serving, is being stored, distributed, held for service, used in preparation of a menu item, or served.',
 'Accurate thermometer not provided in refrigerated or hot holding equipment.',
 'Appropriately scaled metal stem-type thermometer or thermocouple not provided or used to evaluate temperatures of potentially hazardous foods during cooking, cooling, reheating and holding.',
 'Ashtray present in smoke-free area.',
 'Bulb not shielded or shatterproof, in areas where there is extreme heat, temperature changes, or where accidental contact may occur.',
 'Caloric content not posted on menus, menu boards or food tags, in a food service establishment that is 1 of 15 or more outlets operating the same type of business nationally 

In [ ]:
# mice, rats, vermin